<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/6_proyectos/clase_41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversación con grok para ejercicio de detección de victimas y victimarios

https://grok.com/share/bGVnYWN5_c75023c4-13a0-493f-a185-82dadf1d0c80


¡Entendido! Quieres ejemplos de código usando MoveNet para procesar videos y detectar poses, lo cual es útil para identificar agresores y víctimas en tu caso de detección de crímenes en videos. MoveNet es un modelo ligero de estimación de poses de Google, ideal para analizar skeletons (puntos clave del cuerpo) en tiempo real, y puede ayudarte a inferir roles basados en movimientos (ej. un agresor levantando un brazo para golpear, una víctima retrocediendo). A continuación, te doy dos ejemplos prácticos en Python usando MoveNet: uno para procesar un solo frame (imagen) y otro para videos, integrando detección de personas y análisis básico de roles.

### Prerrequisitos
- **Instalaciones**:
  ```bash
  pip install tensorflow tensorflow-hub opencv-python numpy
  ```
- **Hardware**: MoveNet es ligero, funciona en CPU, pero GPU acelera (Colab es buena opción).
- **Modelo**: Usaremos MoveNet.SinglePose (versión "Lightning" para rapidez o "Thunder" para precisión). Disponible en TensorFlow Hub.

### Ejemplo 1: Detección de Poses en una Imagen
Este código detecta puntos clave (keypoints) en una imagen estática, útil para entender cómo MoveNet procesa una persona.

```python
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Carga MoveNet desde TensorFlow Hub (Lightning para rapidez)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

# Preprocesa imagen
def preprocess_image(image):
    img = cv2.resize(image, (192, 192))  # MoveNet Lightning usa 192x192
    img = tf.convert_to_tensor(img, dtype=tf.int32)
    img = tf.expand_dims(img, axis=0)  # Añade batch dimension
    return img

# Detecta poses
def detect_pose(image):
    input_image = preprocess_image(image)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # [persona, keypoints, (y, x, confianza)]
    return keypoints

# Visualiza keypoints
def draw_keypoints(image, keypoints, conf_threshold=0.3):
    height, width = image.shape[:2]
    for keypoint in keypoints:
        y, x, confidence = keypoint
        if confidence > conf_threshold:
            x = int(x * width)
            y = int(y * height)
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Dibuja punto verde
    return image

# Carga y procesa imagen
image_path = 'tu_imagen.jpg'  # Reemplaza con tu imagen
image = cv2.imread(image_path)
keypoints = detect_pose(image)
image_with_keypoints = draw_keypoints(image.copy(), keypoints)

# Guarda o muestra resultado
cv2.imwrite('output_image.jpg', image_with_keypoints)
cv2.imshow('Pose Detection', image_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

**Explicación**:
- Carga MoveNet desde TensorFlow Hub.
- Procesa la imagen (resize a 192x192, formato requerido por MoveNet).
- Extrae 17 keypoints (nariz, hombros, codos, muñecas, etc.) con coordenadas (y, x) y confianza.
- Dibuja puntos en la imagen si la confianza es >0.3.
- Útil para pruebas iniciales. Para roles (agresor/víctima), analiza keypoints como distancia entre manos y torso de otra persona.

### Ejemplo 2: Detección de Poses en Video con Análisis de Roles
Este código procesa un video, detecta personas con YOLOv8 (para identificar múltiples actores), aplica MoveNet para estimar poses y hace un análisis básico de movimientos para inferir agresor/víctima.

```python
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from ultralytics import YOLO  # Para YOLOv8

# Carga modelos
movenet_model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = movenet_model.signatures['serving_default']
yolo = YOLO('yolov8n.pt')  # Modelo YOLOv8 preentrenado (instala ultralytics: pip install ultralytics)

# Preprocesa frame para MoveNet
def preprocess_frame(frame):
    img = cv2.resize(frame, (192, 192))
    img = tf.convert_to_tensor(img, dtype=tf.int32)
    img = tf.expand_dims(img, axis=0)
    return img

# Detecta poses
def detect_pose(frame):
    input_image = preprocess_frame(frame)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # [persona, keypoints, (y, x, conf)]
    return keypoints

# Analiza movimiento para roles (heurística simple)
def analyze_roles(keypoints_list, person_boxes):
    roles = []
    for i, (keypoints, box) in enumerate(zip(keypoints_list, person_boxes)):
        # Ejemplo: Si mano derecha (keypoint 10) está alta y cerca de otra persona, posible agresor
        right_wrist = keypoints[10]  # [y, x, conf]
        if right_wrist[2] > 0.3:  # Confianza suficiente
            wrist_y, wrist_x = right_wrist[0], right_wrist[1]
            for j, other_box in enumerate(person_boxes):
                if i != j:  # Evita compararse consigo mismo
                    ox, oy, ow, oh = other_box
                    if abs(wrist_x * 192 - ox) < 50 and wrist_y * 192 < oy:  # Mano cerca y alta
                        roles.append((i, 'agresor'))
                        roles.append((j, 'victima'))
                        return roles
    return roles

# Procesa video
def process_video(video_path, output_path='output_video.mp4'):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detecta personas con YOLOv8
        results = yolo(frame)
        person_boxes = []
        for result in results:
            for box in result.boxes:
                if box.cls == 0:  # 0 es 'person' en COCO
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    person_boxes.append((x1, y1, x2 - x1, y2 - y1))

        # Aplica MoveNet a cada persona
        keypoints_list = []
        for box in person_boxes:
            x, y, w, h = [int(v) for v in box]
            person_crop = frame[y:y+h, x:x+w]
            if person_crop.size > 0:
                keypoints = detect_pose(person_crop)
                keypoints_list.append(keypoints)

        # Analiza roles
        roles = analyze_roles(keypoints_list, person_boxes)

        # Dibuja resultados
        for i, (box, keypoints) in enumerate(zip(person_boxes, keypoints_list)):
            x, y, w, h = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            role = next((r for idx, r in roles if idx == i), 'desconocido')
            cv2.putText(frame, role, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            # Dibuja keypoints
            for kp in keypoints:
                if kp[2] > 0.3:  # Confianza
                    kx, ky = int(kp[1] * w + x), int(kp[0] * h + y)
                    cv2.circle(frame, (kx, ky), 5, (255, 0, 0), -1)

        out.write(frame)

    cap.release()
    out.release()

# Uso
process_video('tu_video.mp4', 'output_video.mp4')
```

**Explicación**:
- **YOLOv8**: Detecta personas y extrae bounding boxes.
- **MoveNet**: Estima poses (17 keypoints) para cada persona detectada.
- **Análisis de Roles**: Heurística simple: si una mano está alta y cerca de otra persona, se etiqueta como "agresor". La otra como "víctima". Puedes mejorar esto con:
  - Modelos ST-GCN para analizar secuencias de poses.
  - Detección de velocidad (optical flow) para confirmar movimientos agresivos.
- **Salida**: Video con bounding boxes, keypoints y etiquetas de roles.

### 5. **Mejoras y Notas**
- **Precisión de Roles**: La heurística es básica. Para mejor precisión:
  - Entrena un modelo (ej. ST-GCN o SlowFast) con datasets como RWF-2000, anotando agresores/víctimas.
  - Usa optical flow (cv2.calcOpticalFlowFarneback) para detectar dirección/velocidad de movimientos.
- **Escalabilidad**: MoveNet es rápido (10-20 FPS en CPU). Para tiempo real, optimiza con TensorRT.
- **Datos**: Fine-tunea con datasets como SCVD o UCF-Crime, anotando roles. Herramientas como CVAT ayudan.
- **Ética**: Anonimiza rostros (ej. con Mediapipe Face Detection) para privacidad.
- **Audio**: Si los videos tienen audio, combina con diarización (Librosa o pyannote.audio) para contexto (ej. gritos).

### 6. **Siguientes Pasos**
- **Fine-Tuning**: Anota un subconjunto de tus videos con CVAT y entrena un modelo custom (ej. en Kinetics o RWF-2000).
- **Modelos Avanzados**: Explora VideoMAE o TimeSformer para acción y roles end-to-end.
- **Código Adicional**: Si quieres integrar optical flow o ST-GCN, puedo dar otro snippet. ¿Detalles sobre tus videos (duración, resolución, tipo de crímenes)? ¿O prefieres enfocarte en algo específico (ej. datasets, entrenamiento)? ¡Dime!

In [1]:
!pip install tensorflow tensorflow-hub opencv-python-headless ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.2 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from ultralytics import YOLO  # Para YOLOv8

# Carga modelos
movenet_model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = movenet_model.signatures['serving_default']
yolo = YOLO('yolov8n.pt')  # Modelo YOLOv8 preentrenado (instala ultralytics: pip install ultralytics)

# Preprocesa frame para MoveNet
def preprocess_frame(frame):
    img = cv2.resize(frame, (192, 192))
    img = tf.convert_to_tensor(img, dtype=tf.int32)
    img = tf.expand_dims(img, axis=0)
    return img

# Detecta poses
def detect_pose(frame):
    input_image = preprocess_frame(frame)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # [persona, keypoints, (y, x, conf)]
    return keypoints

# Analiza movimiento para roles (heurística simple)
def analyze_roles(keypoints_list, person_boxes):
    roles = []
    for i, (keypoints, box) in enumerate(zip(keypoints_list, person_boxes)):
        # Ejemplo: Si mano derecha (keypoint 10) está alta y cerca de otra persona, posible agresor
        right_wrist = keypoints[10]  # [y, x, conf]
        if right_wrist[2] > 0.3:  # Confianza suficiente
            wrist_y, wrist_x = right_wrist[0], right_wrist[1]
            for j, other_box in enumerate(person_boxes):
                if i != j:  # Evita compararse consigo mismo
                    ox, oy, ow, oh = other_box
                    if abs(wrist_x * 192 - ox) < 50 and wrist_y * 192 < oy:  # Mano cerca y alta
                        roles.append((i, 'agresor'))
                        roles.append((j, 'victima'))
                        return roles
    return roles

# Procesa video
def process_video(video_path, output_path='output_video.mp4'):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detecta personas con YOLOv8
        results = yolo(frame)
        person_boxes = []
        for result in results:
            for box in result.boxes:
                if box.cls == 0:  # 0 es 'person' en COCO
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    person_boxes.append((x1, y1, x2 - x1, y2 - y1))

        # Aplica MoveNet a cada persona
        keypoints_list = []
        for box in person_boxes:
            x, y, w, h = [int(v) for v in box]
            person_crop = frame[y:y+h, x:x+w]
            if person_crop.size > 0:
                keypoints = detect_pose(person_crop)
                keypoints_list.append(keypoints)

        # Analiza roles
        roles = analyze_roles(keypoints_list, person_boxes)

        # Dibuja resultados
        for i, (box, keypoints) in enumerate(zip(person_boxes, keypoints_list)):
            x, y, w, h = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            role = next((r for idx, r in roles if idx == i), 'desconocido')
            cv2.putText(frame, role, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            # Dibuja keypoints
            for kp in keypoints:
                if kp[2] > 0.3:  # Confianza
                    kx, ky = int(kp[1] * w + x), int(kp[0] * h + y)
                    cv2.circle(frame, (kx, ky), 5, (255, 0, 0), -1)

        out.write(frame)

    cap.release()
    out.release()

# Uso
process_video('tu_video.mp4', 'output_video.mp4')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Instalar dependencias
!pip install tensorflow tensorflow-hub opencv-python-headless ultralytics

import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from ultralytics import YOLO
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from google.colab import files

# Carga modelos
movenet_model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = movenet_model.signatures['serving_default']
yolo = YOLO('yolov8n.pt')  # Modelo YOLOv8 preentrenado

# Preprocesa frame para MoveNet
def preprocess_frame(frame):
    img = cv2.resize(frame, (192, 192))
    img = tf.convert_to_tensor(img, dtype=tf.int32)
    img = tf.expand_dims(img, axis=0)
    return img

# Detecta poses
def detect_pose(frame):
    input_image = preprocess_frame(frame)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # [persona, keypoints, (y, x, conf)]
    return keypoints

# Analiza movimiento para roles (heurística simple)
def analyze_roles(keypoints_list, person_boxes):
    roles = []
    for i, (keypoints, box) in enumerate(zip(keypoints_list, person_boxes)):
        right_wrist = keypoints[10]  # [y, x, conf]
        if right_wrist[2] > 0.3:  # Confianza suficiente
            wrist_y, wrist_x = right_wrist[0], right_wrist[1]
            for j, other_box in enumerate(person_boxes):
                if i != j:  # Evita compararse consigo mismo
                    ox, oy, ow, oh = other_box
                    if abs(wrist_x * 192 - ox) < 50 and wrist_y * 192 < oy:  # Mano cerca y alta
                        roles.append((i, 'agresor'))
                        roles.append((j, 'victima'))
                        return roles
    return roles

# Calcula métricas (F1-score para roles)
def calculate_metrics(true_roles, pred_roles):
    y_true = [r for _, r in true_roles]
    y_pred = [r for _, r in pred_roles]
    if len(y_true) == len(y_pred):
        f1 = f1_score(y_true, y_pred, labels=['agresor', 'victima'], average='weighted')
        return f1
    return 0.0

# Procesa video
def process_video(video_path, output_path='output_video.mp4', ground_truth_roles=None):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_count = 0
    all_pred_roles = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detecta personas con YOLOv8
        results = yolo(frame)
        person_boxes = []
        for result in results:
            for box in result.boxes:
                if box.cls == 0:  # 0 es 'person' en COCO
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    person_boxes.append((x1, y1, x2 - x1, y2 - y1))

        # Aplica MoveNet a cada persona
        keypoints_list = []
        for box in person_boxes:
            x, y, w, h = [int(v) for v in box]
            person_crop = frame[y:y+h, x:x+w]
            if person_crop.size > 0:
                keypoints = detect_pose(person_crop)
                keypoints_list.append(keypoints)

        # Analiza roles
        roles = analyze_roles(keypoints_list, person_boxes)
        all_pred_roles.append(roles)

        # Dibuja resultados
        for i, (box, keypoints) in enumerate(zip(person_boxes, keypoints_list)):
            x, y, w, h = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            role = next((r for idx, r in roles if idx == i), 'desconocido')
            cv2.putText(frame, role, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            for kp in keypoints:
                if kp[2] > 0.3:
                    kx, ky = int(kp[1] * w + x), int(kp[0] * h + y)
                    cv2.circle(frame, (kx, ky), 5, (255, 0, 0), -1)

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

    # Calcula métricas si hay ground truth
    if ground_truth_roles:
        f1 = calculate_metrics(ground_truth_roles, all_pred_roles[0])  # Ejemplo con un frame
        print(f"F1-Score para roles: {f1:.4f}")

    return output_path

# Sube el video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# Ground truth de ejemplo (ajusta con tus datos reales)
ground_truth_roles = [(0, 'agresor'), (1, 'victima')]  # Asumiendo 2 personas en el video

# Procesa el video y descarga el resultado
output_path = process_video(video_path, 'output_video.mp4', ground_truth_roles)
files.download(output_path)

Saving people-detection.mp4 to people-detection.mp4

0: 384x640 (no detections), 77.4ms
Speed: 15.4ms preprocess, 77.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.4ms
Speed: 3.4ms preprocess, 6.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.2ms
Speed: 2.7ms preprocess, 6.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.0ms
Speed: 2.7ms preprocess, 6.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.0ms
Speed: 2.3ms preprocess, 6.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.9ms
Speed: 2.3ms preprocess, 5.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.2ms
Speed: 2.5ms preprocess, 7.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections),

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>